In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
def readin_large_data():
    """Read small data set into a 2D numpy array
    return: numpy array contains the small data set, shape: (50,000, 230)
    """
    PATH = os.getcwd()
    DATA_PATH = os.path.join(PATH, 'large_clean.csv')
    
    data = pd.read_csv(DATA_PATH, delimiter=',')
    
    del data['Unnamed: 0']
    
    return data

In [3]:
data = readin_large_data()
print(data)

                  0             1             2             3             4  \
0      2.851628e-05  1.141128e-06  2.000173e-05  9.344759e-05  2.233666e-05   
1      2.041817e-04 -1.603930e-04  4.684614e-05  8.388534e-05 -1.164747e-04   
2      1.666764e-04 -8.416159e-06  1.386948e-04  3.087744e-04  5.063547e-05   
3      4.278522e-05  3.384583e-06  2.402150e-05  1.092209e-04  4.867743e-05   
4      1.405754e-06  5.865534e-07  3.127575e-06  8.438816e-06 -1.161525e-06   
5      2.644090e-06  3.518705e-07  5.269246e-07  8.094414e-06  1.074261e-06   
6      6.092146e-05 -1.267638e-05  4.985303e-05  9.458280e-05  1.146054e-05   
7      1.377391e-05  6.094651e-06 -2.543359e-07  3.963860e-05 -9.475450e-06   
8      8.845893e-06  4.884442e-07  6.580806e-06  3.016162e-05 -6.050665e-06   
9      9.388848e-05 -1.538702e-05  1.170401e-04  4.909937e-04 -2.697568e-04   
10     1.335293e-05  9.619302e-07  2.783564e-05  7.844316e-05 -4.996356e-05   
11     4.202320e-04 -3.023209e-04  5.283924e-05  1.5

In [4]:
def readin_label(target):
    """Read upselling, churn or appetency label into a 2D numpy array
    parameter target: choose which label to load
    return: numpy array contains the specified label of small data set, shape: (50,000, 1)
    """
    assert target in ['upselling', 'churn', 'appetency']
    PATH = os.getcwd()
    LABEL_PATH = os.path.join(PATH, 'data_large','orange_large_train_'+target+'.labels')

    label = pd.read_csv(LABEL_PATH, delimiter='\t', header=None)
    
    label[label==1] = True
    label[label==-1] = False
    
    return label

upselling_label = readin_label('upselling')
print(upselling_label.shape)
churn_label = readin_label('churn')
print(churn_label.shape)
appetency_label = readin_label('appetency')
print(appetency_label.shape)

(50000, 1)
(50000, 1)
(50000, 1)


In [5]:
labels = np.hstack([appetency_label, churn_label, upselling_label])
labels = np.hstack([labels, ~np.any(labels, axis=1)[:,np.newaxis]])
labels = labels.astype(int)

In [7]:
print(labels.shape)

(50000, 4)


In [13]:
import sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
import keras
from sklearn.cross_validation import train_test_split
from keras import regularizers

model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=1443, kernel_regularizer=regularizers.l2(0.05)))
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
model.add(Dense(units=100, activation='relu', kernel_regularizer=regularizers.l2(0.05)))
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
model.add(Dense(units=100, activation='relu', kernel_regularizer=regularizers.l2(0.05)))
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
# model.add(Dense(units=100, activation='relu'))
# keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
# model.add(Dense(units=100, activation='relu'))
# keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

model.add(Dense(units=4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

# trainX, valX, trainY, valY = train_test_split(clean_numerical_data, labels, test_size=0.2, random_state=42)



model.fit(sklearn.preprocessing.robust_scale(data), labels, epochs=400, batch_size=200, validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/400
40000/40000 [==============================] - 3s 66us/step - loss: 16.4419 - acc: 0.8052 - val_loss: 13.0014 - val_acc: 0.8366
Epoch 2/400
40000/40000 [==============================] - 2s 62us/step - loss: 10.6255 - acc: 0.8343 - val_loss: 8.5754 - val_acc: 0.8369
Epoch 3/400
40000/40000 [==============================] - 3s 63us/step - loss: 7.1365 - acc: 0.8347 - val_loss: 5.8895 - val_acc: 0.8367
Epoch 4/400
40000/40000 [==============================] - 3s 63us/step - loss: 4.9799 - acc: 0.8348 - val_loss: 4.1889 - val_acc: 0.8366
Epoch 5/400
40000/40000 [==============================] - 3s 66us/step - loss: 3.5852 - acc: 0.8349 - val_loss: 3.0586 - val_acc: 0.8369
Epoch 6/400
40000/40000 [==============================] - 2s 62us/step - loss: 2.6428 - acc: 0.8351 - val_loss: 2.2809 - val_acc: 0.8371
Epoch 7/400
40000/40000 [==============================] - 3s 64us/step - loss: 1.9914 - acc: 0.8350 - val_loss: 1.7425

Epoch 118/400
40000/40000 [==============================] - 3s 63us/step - loss: 0.5595 - acc: 0.8531 - val_loss: 0.6146 - val_acc: 0.8475
Epoch 119/400
40000/40000 [==============================] - 2s 62us/step - loss: 0.5589 - acc: 0.8530 - val_loss: 0.6149 - val_acc: 0.8461
Epoch 120/400
40000/40000 [==============================] - 3s 64us/step - loss: 0.5584 - acc: 0.8541 - val_loss: 0.6157 - val_acc: 0.8448
Epoch 121/400
40000/40000 [==============================] - 2s 62us/step - loss: 0.5588 - acc: 0.8537 - val_loss: 0.6171 - val_acc: 0.8490
Epoch 122/400
40000/40000 [==============================] - 3s 64us/step - loss: 0.5583 - acc: 0.8539 - val_loss: 0.6162 - val_acc: 0.8446
Epoch 123/400
40000/40000 [==============================] - 3s 63us/step - loss: 0.5577 - acc: 0.8548 - val_loss: 0.6173 - val_acc: 0.8462
Epoch 124/400
40000/40000 [==============================] - 2s 61us/step - loss: 0.5569 - acc: 0.8549 - val_loss: 0.6177 - val_acc: 0.8477
Epoch 125/400
40000/

40000/40000 [==============================] - 2s 61us/step - loss: 0.5241 - acc: 0.8653 - val_loss: 0.6822 - val_acc: 0.8458
Epoch 235/400
40000/40000 [==============================] - 3s 63us/step - loss: 0.5237 - acc: 0.8650 - val_loss: 0.6817 - val_acc: 0.8452
Epoch 236/400
40000/40000 [==============================] - 3s 64us/step - loss: 0.5240 - acc: 0.8643 - val_loss: 0.6826 - val_acc: 0.8453
Epoch 237/400
40000/40000 [==============================] - 3s 65us/step - loss: 0.5237 - acc: 0.8653 - val_loss: 0.6806 - val_acc: 0.8459
Epoch 238/400
40000/40000 [==============================] - 2s 62us/step - loss: 0.5224 - acc: 0.8652 - val_loss: 0.6851 - val_acc: 0.8480
Epoch 239/400
40000/40000 [==============================] - 3s 63us/step - loss: 0.5231 - acc: 0.8655 - val_loss: 0.6834 - val_acc: 0.8462
Epoch 240/400
40000/40000 [==============================] - 3s 63us/step - loss: 0.5218 - acc: 0.8650 - val_loss: 0.6877 - val_acc: 0.8460
Epoch 241/400
40000/40000 [=======

40000/40000 [==============================] - 3s 63us/step - loss: 0.4938 - acc: 0.8698 - val_loss: 0.7731 - val_acc: 0.8381
Epoch 351/400
40000/40000 [==============================] - 2s 57us/step - loss: 0.4938 - acc: 0.8699 - val_loss: 0.7732 - val_acc: 0.8394
Epoch 352/400
40000/40000 [==============================] - 2s 58us/step - loss: 0.4970 - acc: 0.8697 - val_loss: 0.7740 - val_acc: 0.8410
Epoch 353/400
40000/40000 [==============================] - 3s 64us/step - loss: 0.4944 - acc: 0.8703 - val_loss: 0.7729 - val_acc: 0.8369
Epoch 354/400
40000/40000 [==============================] - 2s 62us/step - loss: 0.4941 - acc: 0.8705 - val_loss: 0.7800 - val_acc: 0.8399
Epoch 355/400
40000/40000 [==============================] - 3s 64us/step - loss: 0.4940 - acc: 0.8703 - val_loss: 0.7761 - val_acc: 0.8368
Epoch 356/400
40000/40000 [==============================] - 2s 62us/step - loss: 0.4924 - acc: 0.8706 - val_loss: 0.7777 - val_acc: 0.8396
Epoch 357/400
40000/40000 [=======